In [1]:
# Data Processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Modelling
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
import joblib

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [2]:
rf = joblib.load("./random_forest.joblib")

In [6]:
data = pd.read_excel('Data_Atlanta_West.xlsx', sheet_name='Sheet1')

In [3]:
def simulate_demand(columns, entry, rate_day, rate_mile, demand_forecasting_model):
    # Assuming you have a model that predicts demand based on price and weather
    # Replace this with your actual demand forecasting model
    entry['RATE DAY'] = rate_day
    entry['RATE MILE'] = rate_mile
    entry_array = entry.values.reshape(1, -1)
    i=[0,4,5,1,2,3]
    entry_array = entry_array[:,i]
    entry_df = pd.DataFrame(entry_array, columns=columns)
    predicted_demand = demand_forecasting_model.predict(entry_df)

    # You might have more complex logic here, but for demonstration purposes:
    estimated_demand = predicted_demand[0]  # Assuming the model predicts a single value
    return estimated_demand

In [4]:
def optimize_prices(columns, transaction, initial_demand, initial_day, initial_mile, max_iterations=10, day_step=.5, mile_step=0.01, demand_threshold=0.92, lower_demand_threshold=0.70):
    best_prices = (initial_day, initial_mile)
    best_demand = initial_demand

    for day_rate in np.arange(initial_day - max_iterations * day_step, initial_day + max_iterations * day_step, day_step):
        for mile_rate in np.arange(initial_mile - max_iterations * mile_step, initial_mile + max_iterations * mile_step, mile_step):
            demand = simulate_demand(columns, transaction, day_rate, mile_rate, rf)
            print(demand, day_rate, mile_rate)
            if initial_demand <= demand <= demand_threshold and demand > best_demand:
                best_demand = demand
                best_prices = (day_rate, mile_rate)

    return best_prices, best_demand

In [8]:
columns = data.columns
# for x in range(10):
#     isolated_row = test_features.iloc[x].drop(['RATE DAY', 'RATE MILE'])
#     demand = test_labels.iloc[x]
#     print(f"Isolated Row {test_features.iloc[x]}, {test_labels.iloc[x]}")
#     optimal_price, optimal_demand = optimize_prices(columns, isolated_row, demand, test_features.iloc[x]['RATE DAY'],  test_features.iloc[x]['RATE MILE'])
#     print(f"Optimal Price: {optimal_price}, Corresponding Demand: {optimal_demand}")

isolated_row = data.iloc[0].drop(['RATE DAY', 'RATE MILE'])
demand = data.iloc[0]
print(f"Isolated Row {data.iloc[0]}, {data.iloc[0]}")
optimal_price, optimal_demand = optimize_prices(columns, isolated_row, demand, data.iloc[0]['RATE DAY'],  data.iloc[0]['RATE MILE'])
print(f"Optimal Price: {optimal_price}, Corresponding Demand: {optimal_demand}")

Isolated Row DATE OUT                2015-01-16 00:00:00
TIME OUT                           12:55:00
TIME IN                            12:30:00
DATE IN                 2015-01-17 00:00:00
RATE DAY                              46.99
RATE WEEK                            328.93
RATE MILE                              0.99
MILES USED                               69
FUEL LOST                                 4
DAMAGED                                   0
DURATION                                  1
MILEAGE PRICE                         68.31
BOARD PRICE                           46.99
EXP PRICE                             115.3
GROUP                                     3
SumOfFLEET Gas                           39
SumOfFLEET Parcel                        69
SumOfFLEET All                          109
Utilization Rate All               0.706422
Name: 0, dtype: object, DATE OUT                2015-01-16 00:00:00
TIME OUT                           12:55:00
TIME IN                            12:3

ValueError: Shape of passed values is (1, 6), indices imply (1, 19)